In [54]:
# Dependencies
import matplotlib.pyplot as plt
import requests
import pandas as pd
from pprint import pprint

In [55]:
# Save config information.
url = "https://data.ca.gov/api/3/action/datastore_search?resource_id=d73ee828-c2c6-485c-91dc-c26bd9ce3991" # base url

# Build partial query URL
query_url = f"{url}&limit=1000"
response = requests.get(query_url).json()
records = response['result']['records']

# put records in dataframe
records_df = pd.DataFrame(records)
records_df.head()

,_id,OBJECTID,Year,State,Agency,Unit ID,Fire Name,Local Incident Number,Alarm Date,Containment Date,...,Collection Method,Management Objective,GIS Calculated Acres,Comments,Complex Name,IRWIN ID,Fire Number (historical use),Complex ID,Shape__Area,Shape__Length
0,1,1,2023,CA,CDF,SKU,WHITWORTH,00004808,6/17/2023 12:00:00 AM,6/17/2023 12:00:00 AM,...,1,1,5.729125,None,None,{7985848C-0AC2-4BA4-8F0E-29F778652E61},None,None,41407.83984375,1247.16603398886
1,2,2,2023,CA,LRA,BTU,KAISER,00010225,6/2/2023 12:00:00 AM,6/2/2023 12:00:00 AM,...,1,1,13.60238,None,None,{43EBCC88-B3AC-48EB-8EF5-417FE0939CCF},None,None,93455.87890625,1285.51454971661
2,3,3,2023,CA,CDF,AEU,JACKSON,00017640,7/1/2023 12:00:00 AM,7/2/2023 12:00:00 AM,...,1,1,27.81446,None,None,{B64E1355-BF1D-441A-95D0-BC1FBB93483B},None,None,183028.5,2697.58742917456
3,4,4,2023,CA,CDF,AEU,CARBON,00018821,7/11/2023 12:00:00 AM,7/11/2023 12:00:00 AM,...,1,1,58.76023,None,None,{CB41DB0A-E4B1-489D-A4EA-738F2CD6DB3B},None,None,388221.953125,2548.73877855158
4,5,5,2023,CA,CDF,AEU,LIBERTY,00018876,7/11/2023 12:00:00 AM,7/12/2023 12:00:00 AM,...,1,1,70.979,None,None,{F83F70A4-07A7-40B8-BD51-10CCC1C30D63},None,None,466456.0078125,5106.93652582734


In [56]:
# drop first two columns
records_df = records_df.drop(columns=['_id', 'OBJECTID'])

# rename columns
renamed_df = records_df.rename(columns={'GIS Calculated Acres': 'Acres', 'Shape__Area': 'Area', 'Shape__Length':'Perimeter'})

# reorder columns
new_order = ['Year', 'Fire Name', 'State', 'Alarm Date', 'Containment Date', 'Cause',
       'Collection Method', 'Management Objective', 'Acres', 'Comments',
       'Complex Name', 'IRWIN ID', 'Fire Number (historical use)',
       'Complex ID', 'Area', 'Perimeter', 'Local Incident Number', 'Agency', 'Unit ID', ]

df_reordered = renamed_df[new_order]

In [57]:
# Convert 'Alarm Date' and 'Containment Date' columns to datetime format
df_reordered['Alarm Date'] = pd.to_datetime(df_reordered['Alarm Date'])
df_reordered['Containment Date'] = pd.to_datetime(df_reordered['Containment Date'])

# Extract date part only
df_reordered['Alarm Date'] = df_reordered['Alarm Date'].dt.date
df_reordered['Containment Date'] = df_reordered['Containment Date'].dt.date

# Convert 'Alarm Date' and 'Containment Date' columns back to datetime format
df_reordered['Alarm Date'] = pd.to_datetime(df_reordered['Alarm Date'])
df_reordered['Containment Date'] = pd.to_datetime(df_reordered['Containment Date'])

# Calculate duration in days and add as a new column
df_reordered['Fire Duration (Days)'] = (df_reordered['Containment Date'] - df_reordered['Alarm Date']).dt.days

new_order = ['Year', 'Fire Name', 'State', 'Alarm Date', 'Containment Date', 'Fire Duration (Days)', 'Cause',
       'Collection Method', 'Management Objective', 'Acres', 'Comments',
       'Complex Name', 'IRWIN ID', 'Fire Number (historical use)',
       'Complex ID', 'Area', 'Perimeter', 'Local Incident Number', 'Agency', 'Unit ID', ]

df_reordered = df_reordered[new_order]

C:\Users\jenni\AppData\Local\Temp\ipykernel_6020\2344807503.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_reordered['Alarm Date'] = pd.to_datetime(df_reordered['Alarm Date'])
C:\Users\jenni\AppData\Local\Temp\ipykernel_6020\2344807503.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_reordered['Containment Date'] = pd.to_datetime(df_reordered['Containment Date'])


In [58]:
# Define the cause mapping dictionary
cause_mapping = {
    '1': 'Lightning',
    '2': 'Equipment Use',
    '3': 'Smoking',
    '4': 'Campfire',
    '5': 'Debris',
    '6': 'Railroad',
    '7': 'Arson',
    '8': 'Playing with fire',
    '9': 'Miscellaneous',
    '10': 'Vehicle',
    '11': 'Powerline',
    '12': 'Firefighter Training',
    '13': 'Non-Firefighter Training',
    '14': 'Unknown / Unidentified',
    '15': 'Structure',
    '16': 'Aircraft',
    '17': '',
    '18': 'Escaped Prescribed Burn',
    '19': 'Illegal Alien Campfire'
}

# replace cause values with causes
df_reordered['Cause'] = df_reordered['Cause'].replace(cause_mapping)

df_reordered.head()

,Year,Fire Name,State,Alarm Date,Containment Date,Fire Duration (Days),Cause,Collection Method,Management Objective,Acres,Comments,Complex Name,IRWIN ID,Fire Number (historical use),Complex ID,Area,Perimeter,Local Incident Number,Agency,Unit ID
0,2023,WHITWORTH,CA,2023-06-17,2023-06-17,0.0,Debris,1,1,5.729125,None,None,{7985848C-0AC2-4BA4-8F0E-29F778652E61},None,None,41407.83984375,1247.16603398886,00004808,CDF,SKU
1,2023,KAISER,CA,2023-06-02,2023-06-02,0.0,Debris,1,1,13.60238,None,None,{43EBCC88-B3AC-48EB-8EF5-417FE0939CCF},None,None,93455.87890625,1285.51454971661,00010225,LRA,BTU
2,2023,JACKSON,CA,2023-07-01,2023-07-02,1.0,Equipment Use,1,1,27.81446,None,None,{B64E1355-BF1D-441A-95D0-BC1FBB93483B},None,None,183028.5,2697.58742917456,00017640,CDF,AEU
3,2023,CARBON,CA,2023-07-11,2023-07-11,0.0,Miscellaneous,1,1,58.76023,None,None,{CB41DB0A-E4B1-489D-A4EA-738F2CD6DB3B},None,None,388221.953125,2548.73877855158,00018821,CDF,AEU
4,2023,LIBERTY,CA,2023-07-11,2023-07-12,1.0,Unknown / Unidentified,1,1,70.979,None,None,{F83F70A4-07A7-40B8-BD51-10CCC1C30D63},None,None,466456.0078125,5106.93652582734,00018876,CDF,AEU


In [59]:
# create dataframes based on year of alarm dates
df_reordered['Alarm Date'] = pd.to_datetime(df_reordered['Alarm Date'])

df_2020 = df_reordered[df_reordered['Alarm Date'].dt.year == 2020]
df_2021 = df_reordered[df_reordered['Alarm Date'].dt.year == 2021]
df_2022 = df_reordered[df_reordered['Alarm Date'].dt.year == 2022]
df_2023 = df_reordered[df_reordered['Alarm Date'].dt.year == 2023]
#df_2024 = df_reordered[df_reordered['Alarm Date'].dt.year == 2024]

In [60]:
# sort the dataframes by alarm date
df_2020_sorted = df_2020.sort_values(by='Alarm Date')
df_2021_sorted = df_2021.sort_values(by='Alarm Date')
df_2022_sorted = df_2022.sort_values(by='Alarm Date')
df_2023_sorted = df_2023.sort_values(by='Alarm Date')

In [61]:
df_2020_sorted.head()

,Year,Fire Name,State,Alarm Date,Containment Date,Fire Duration (Days),Cause,Collection Method,Management Objective,Acres,Comments,Complex Name,IRWIN ID,Fire Number (historical use),Complex ID,Area,Perimeter,Local Incident Number,Agency,Unit ID
986,2020,WILLOW,CA,2020-03-10,2020-03-11,1.0,Debris,1,1,33.97389,None,None,{F5DBC85A-D6B6-4376-B793-B72BD4A17DE7},None,None,230549.78125,2477.15181547552,00006064,CDF,NEU
981,2020,FLEMING,CA,2020-03-31,2020-04-01,1.0,Miscellaneous,1,1,12.93155,None,None,None,None,None,86679.4453125,2029.52488137503,00007619,CDF,NEU
982,2020,MELANESE,CA,2020-04-14,2020-04-19,5.0,Escaped Prescribed Burn,1,1,10.31596,None,None,{DDC3C7A0-29A9-4FD9-8FE5-92F2FE31D283},None,None,70179.1328125,1342.74290294001,00008471,CDF,NEU
979,2020,AMORUSO,CA,2020-06-01,2020-06-04,3.0,Equipment Use,1,1,685.585,None,None,{75ACF0A2-FBCF-4BC6-AF99-BB28865C3F95},None,None,4578171.9140625,12399.3753905116,00011799,CDF,NEU
993,2020,RIOSA,CA,2020-06-15,2020-06-15,0.0,Powerline,1,1,13.69472,None,None,{9DF35003-C12B-4717-8819-DA99A4714809},None,None,91856.8515625,1743.67087360593,00012970,CDF,NEU


In [68]:
df_2021_sorted.head()

,Year,Fire Name,State,Alarm Date,Containment Date,Fire Duration (Days),Cause,Collection Method,Management Objective,Acres,Comments,Complex Name,IRWIN ID,Fire Number (historical use),Complex ID,Area,Perimeter,Local Incident Number,Agency,Unit ID
966,2021,MIGUEL 3,CA,2021-01-07,2021-01-07,0.0,Unknown / Unidentified,4,1,21.31999,NS1P,None,None,None,None,122097.42578125,1645.44525529025,00000530,FWS,TNR
749,2021,NEWTON,CA,2021-01-08,2021-01-08,0.0,Unknown / Unidentified,1,1,2.265879,,None,None,None,None,13399.23828125,443.870646367217,00009554,CCO,LAC
952,2021,NEWTON,CA,2021-01-08,2021-01-08,0.0,Unknown / Unidentified,1,1,1.865689,None,None,{D22D5497-4072-4D5B-82E1-A04302266D13},None,None,11034.1484375,782.094252013949,00000001,USF,SMP
720,2021,BORDER,CA,2021-01-10,2021-01-21,11.0,Miscellaneous,7,1,1.850452,None,None,{560903DC-8AFF-4091-9C85-A3B806EDE7BA},None,None,10579.1171875,504.822650768738,00000734,CDF,MVU
721,2021,BORDER 2,CA,2021-01-13,2021-01-21,8.0,Debris,7,1,18.64428,None,None,{B36106D6-22FD-4AB4-9603-C045EC5FAD62},None,None,106607.16796875,1924.43632122754,00000960,CDF,MVU


In [63]:
df_2022_sorted.head()

,Year,Fire Name,State,Alarm Date,Containment Date,Fire Duration (Days),Cause,Collection Method,Management Objective,Acres,Comments,Complex Name,IRWIN ID,Fire Number (historical use),Complex ID,Area,Perimeter,Local Incident Number,Agency,Unit ID
524,2022,PRICE,CA,2022-01-01,2022-01-01,0.0,Campfire,6,1,0.06728618,Transient Campfire Fatality,None,{DEA6CF90-F556-437A-8F57-52F9D06EF8BA},None,None,408.3359375,80.8627515888373,00000021,CDF,SLU
400,2022,COLORADO,CA,2022-01-21,2022-02-02,12.0,Debris,1,1,687.4208,None,None,{AB4FCB89-2F15-4E68-A215-206FD00FD4F1},None,None,4300944.3671875,19831.8990106189,00000419,CDF,BEU
405,2022,EMERALD,CA,2022-02-10,2022-02-19,9.0,Unknown / Unidentified,1,1,154.6207,,None,{D30E9DBE-24F4-462E-A030-8BD771DD26C3},None,None,903499.58203125,6738.43566695963,00019793,CCO,ORC
537,2022,SYCAMORE,CA,2022-02-10,2022-02-11,1.0,Unknown / Unidentified,7,1,3.735543,None,None,{77FBE31A-636F-44C0-ABE7-611D1C06F214},None,None,22052.17578125,1130.49981314723,00051618,CCO,LAC
284,2022,AIRPORT,CA,2022-02-16,2022-02-26,10.0,Unknown / Unidentified,3,1,4140.06,None,None,{7AFEDAC3-F210-4FFC-8F27-99560E4BADF6},None,None,26508134.5898438,85420.8352154112,00002479,CDF,BDU


In [67]:
df_2023_sorted.head()

,Year,Fire Name,State,Alarm Date,Containment Date,Fire Duration (Days),Cause,Collection Method,Management Objective,Acres,Comments,Complex Name,IRWIN ID,Fire Number (historical use),Complex ID,Area,Perimeter,Local Incident Number,Agency,Unit ID
251,2023,Winterhaven,CA,2023-03-11,2023-03-12,1.0,Unknown / Unidentified,8,1,15.25318,None,None,{EF331AAF-C5CE-4A90-A5EF-EF4FA4C783EB},None,None,87511.7265625,1408.50473883927,00000086,BLM,CRD
161,2023,MAIN,CA,2023-04-06,2023-04-06,0.0,Powerline,1,1,54.54718,None,None,{16315364-C684-49FF-9B91-1C40E7B1D128},None,None,350211.78515625,2757.57893534511,00004512,CDF,BDU
235,2023,IMVITE,CA,2023-04-17,2023-04-22,5.0,Vehicle,7,1,62.93432,None,None,{A9697929-F082-4E67-AAEF-939AF82AC609},None,None,394267.3203125,6353.72908639242,00006194,BLM,CDD
252,2023,Laguna,CA,2023-04-20,2023-04-22,2.0,Unknown / Unidentified,6,1,19.89741,None,None,{020C3C0D-5196-453C-A5D9-21A3DDE8CF26},None,None,114327.46484375,2617.93953495502,00000201,BLM,CRD
188,2023,NOB,CA,2023-04-26,2023-05-11,15.0,Unknown / Unidentified,5,1,226.9751,None,None,{BA8BE5C8-6DBE-4E03-97BE-15C258A8CB8B},None,None,1348920.48046875,5930.42818836032,00006515,USF,BDF


In [65]:
# bar chart for causes? 

# start dates for fires per year

# how would we calculate the density of fires? 